
# First a note on Ensemble Learning

The general principle of ensemble methods is to construct a linear combination of some model ﬁtting method, instead of using a single ﬁt of the method. The main principle behind the ensemble model is that a group of weak learners come together to form a strong learner, thus increasing the accuracy of the model.When we try to predict the target variable using any machine learning technique, the main causes of difference in actual and predicted values are noise, variance, and bias. Ensemble helps to reduce these factors (except noise, which is irreducible error). The noise-related error is mainly due to noise in the training data and can't be removed. However, the errors due to bias and variance can be reduced.
The total error can be expressed as follows: 

### Total Error = Bias + Variance + Irreducible Error 

![Imgur](https://imgur.com/q4Egoes.png)

# Why we do Bootstrap re-sampling

Bootstrapping resamples the original dataset with replacement many thousands of times to create simulated datasets. This process involves drawing random samples from the original dataset. Through bootstrapping you are simply taking samples over and over again from the same group of data (your sample data) to estimate how accurate your estimates about the entire population (what really is out there in the real world) is.

If you were to take one sample and make estimates on the real population, you might not be able to estimate how accurate your estimates are - we only have one estimate and have not identified how this estimate varies with different samples that we might have encountered.

### Bootstrap Aggregation (or Bagging for short), is a simple and very powerful ensemble method.

An ensemble method is a technique that combines the predictions from multiple machine learning algorithms together to make more accurate predictions than any individual model.

Bootstrap Aggregation is a general procedure that can be used to reduce the variance for those algorithm that have high variance. An algorithm that has high variance are decision trees, like classification and regression trees (CART).

Decision trees are sensitive to the specific data on which they are trained. If the training data is changed (e.g. a tree is trained on a subset of the training data) the resulting decision tree can be quite different and in turn the predictions can be quite different.

Bagging is the application of the Bootstrap procedure to a high-variance machine learning algorithm, typically decision trees.

Another major drawback associated with the tree classifiers is that they are unstable. That is, a small
change in the training data set can result in a very different tree. The reason for this lies in the
hierarchical nature of the tree classifiers. An error that occurs in a node at a high level of the tree
propagates all the way down to the leaves below it.


And so Bagging (bootstrap aggregating) can reduce the variance and improve the
generalization error performance. The basic idea is to create B variants, X1, X2 , . . . , XB , of the
training set X, using bootstrap techniques, by uniformly sampling from X with replacement. For
each of the training set variants Xi , a tree Ti is constructed. The final decision for the classification
of a given point is in favor of the class predicted by the majority of the subclassifiers Ti , i =
1, 2, . . . , B.

# Finally Principle of Random Forest with Bootstrapping

The below text is taken from [this](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8950481) source

Firstly, using Bootstrap resampling technique, multiple samples are randomly selected from the original training sample set x to generate a new training sample set. Then, multiple decision trees are constructed to form random forest. Finally, the random forest averages the output of each decision tree to determine the final filling result y. Due to the Bootstrap used in the random decision tree generation process, all samples are not used in a decision tree and the unused samples are called out of band (OOB). Through out of band, the accuracy of the tree can be evaluated. The other trees are evaluated according to the principle and finally average the results

![Imgur](https://imgur.com/qj2c3ur.png)

---

### Now our Code Implementations

### There will be some functions that start with the word "grader" ex: grader_sampples(), grader_30().. etc, we should not change those function definition. Every Grader function has to return True.

In [1]:
import numpy as np # importing numpy for numerical computation
from sklearn.datasets import load_boston # here we are using sklearn's boston dataset
from sklearn.metrics import mean_squared_error # importing mean_squared_error metric

import random
from sklearn.tree import DecisionTreeRegressor

boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable

print("x.shape ", x.shape)
print("y.shape ", y.shape)

x.shape  (506, 13)
y.shape  (506,)


In [2]:
x[:5]

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
        6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02,
        1.5300e+01, 3.9690e+02, 4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9690e+02, 9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9283e+02, 4.0300e+00],
       [3.2370e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01,
        6.9980e+00, 4.5800e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02,
        1.8700e+01, 3.9463e+02, 2.9400e+00],
       [6.9050e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01,
        7.1470e+00, 5.4200e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02,
        1.8700e+01, 3.9690e+02, 5.3300e+00]])

# <font color='red'><b>Task 1
</b></font>

*  <font color='blue'><b>Creating samples</b></font><br>
    <b> Randomly create 30 samples from the whole boston data points</b>
    *  Creating each sample: Consider any random 303(60% of 506) data points from whole data set and then replicate any 203 points from the sampled points
    
     For better understanding of this procedure lets check this examples, assume we have 10 data points [1,2,3,4,5,6,7,8,9,10], first we take 6 data points randomly , consider we have selected [4, 5, 7, 8, 9, 3] now we will replicate 4 points from [4, 5, 7, 8, 9, 3], consder they are [5, 8, 3,7] so our final sample will be [4, 5, 7, 8, 9, 3, 5, 8, 3,7]
* <font color='blue'><b> Create 30 samples </b></font>
    *  Note that as a part of the Bagging when we are taking the random samples <b>make sure each of the sample will have different set of columns</b><br>
Ex: Assume we have 10 columns[1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,9 ,10] for the first sample we will select [3, 4, 5, 9, 1, 2] and for the second sample  [7, 9, 1, 4, 5, 6, 2] and so on...
Make sure each sample will have atleast 3 feautres/columns/attributes

### Step - 1 - Creating samples

<font color='Orange'><b>Algorithm</b></font>

![alt text](https://i.imgur.com/BTVYXQ1.jpg/)

### Generating samples

In [4]:
def generating_samples(input_data, target_data): 
  
  # random.choice to generate random indices without replacement
  # selecting 303 random row indices from the input_data, without replacement
  rows_selected = np.random.choice(len(input_data), 303, replace=False)
  
  # now we will replicate 203 points from above selected rows
  # Replacing Rows => Extracting 206 reandom row indices from the abvoe rows_selected
  rows_203_extracted_from_rows_selected = np.random.choice(rows_selected, 203, replace=False)
  
  # Now get 3 to 13 random column indices from input_data
  number_of_columns_to_select = random.randint(3, 13)
  columns_selected = np.array(random.sample(range(0, 13), number_of_columns_to_select ))
  
  sample_data = input_data[rows_selected[:, None], columns_selected]
  
  target_of_sample_data = target_data[rows_selected]
  
  # Now Replication of Data for 203 data points out of 303 selected points
  
  replicated_203_sample_data_points = input_data[rows_203_extracted_from_rows_selected[:, None], columns_selected ]
  
  target_203_replicated_sample_data = target_data[rows_203_extracted_from_rows_selected]
  
  # Concatenating data
  
  final_sample_data = np.vstack((sample_data, replicated_203_sample_data_points ))
  
  final_target_data = np.vstack((target_of_sample_data.reshape(-1, 1), target_203_replicated_sample_data.reshape(-1, 1) ))
  
  return final_sample_data, final_target_data, rows_selected, columns_selected
  

<font color='cyan'> <b> Grader function - 1 </b> </fongt>

In [5]:
def grader_samples(a,b,c,d):
    length = (len(a)==506  and len(b)==506)
    sampled = (len(a)-len(set([str(i) for i in a]))==203)
    rows_length = (len(c)==303)
    column_length= (len(d)>=3)
    assert(length and sampled and rows_length and column_length)
    return True
  
a,b,c,d = generating_samples(x, y)
print(a.shape)
print(b.shape)
print(c.shape)
print(d.shape)
grader_samples(a,b,c,d)

(506, 6)
(506, 1)
(303,)
(6,)


True

### Creating 30 samples

![alt text](https://i.imgur.com/p8eZaWL.jpg)

In [6]:
# Use generating_samples function to create 30 samples 
# store these created samples in a list
list_input_data =[]
list_output_data =[]
list_selected_row= []
list_selected_columns=[]

for i in range (0, 30):
  a, b, c, d = generating_samples(x, y)
  list_input_data.append(a)
  list_output_data.append(b)
  list_selected_row.append(c)
  list_selected_columns.append(d)

<font color='cyan'> <b>Grader function - 2 </b></font>

In [7]:
def grader_30(a):
    assert(len(a)==30 and len(a[0])==506)
    return True
grader_30(list_input_data)

True

# Step - 2 of Task-1


## Building High Variance Models on each of the sample and finding train MSE value


*  Build a regression trees on each of 30 samples.
*  Computed the predicted values of each data point(506 data points) in our corpus.
*  Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{30}\sum_{k=1}^{30}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$
*  Now calculate the $MSE =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$

## Flowchart for Building regression trees

![alt text](https://i.imgur.com/pcXfSmp.png)

In [8]:
list_of_all_models_decision_tree = []
for i in range(0, 30):
  model_i = DecisionTreeRegressor(max_depth=None)
  model_i.fit(list_input_data[i], list_output_data[i])
  list_of_all_models_decision_tree.append(model_i)

## Flowchart for calculating MSE

![alt text](https://i.imgur.com/sPEE618.png)

After getting predicted_y for each data point, we can use sklearns mean_squared_error to calculate the MSE between predicted_y and actual_y.

# Calculating MSE

In [9]:
from sklearn.metrics import mean_squared_error
from statistics import median

array_of_Y = []

for i in range(0, 30):
  data_point_i = x[:, list_selected_columns[i]]
  target_y_i = list_of_all_models_decision_tree[i].predict(data_point_i)
  array_of_Y.append(target_y_i)
  
  
predicted_array_of_target_y = np.array(array_of_Y)
predicted_array_of_target_y = predicted_array_of_target_y.transpose()

# print(predicted_array_of_target_y.shape)

# Now to calculate MSE, first calculate the Median of Predicted Y
# passing axis=1 will make sure the medians are computed along axis=1
median_predicted_y = np.median(predicted_array_of_target_y, axis=1)
median_predicted_y.shape

print("MSE : ", mean_squared_error(y, median_predicted_y ))

MSE :  0.09994218142619152


# Step - 3 of Task-1

### Calculating the OOB score

*  Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{k}\sum_{\text{k= model which was buit on samples not included } x^{i}}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.
*  Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.


### Given a single query point predict the price of house

Consider xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60] 
Predict the house price for this point as mentioned in the step 2 of Task 1.

## Flowchart for calculating OOB score

![alt text](https://i.imgur.com/95S5Mtm.png)

## Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.

In [10]:
# First noting that our earlier definded variable list_selected_row and list_selected_columns 
# which has the list of selected rows and columns
# e.g. list_selected_row is a 2D array of the form [[], [], []...] each inner-array represnting selected row numbers
# for a specific sample. and len(list_selected_row) is 30 reprsenting the 30 samples we have selected for bootstrapping
# print("list_selected_row[10] ", list_selected_row[10])
# print(list_selected_columns)

y_predicted_oob_median_list = []

for i in range(0, 506):
  indices_for_oob_models = []
  
  # For each of i-th row I shall build a list, of sample size 30
  # ONLY condition being that this i-th row should not be part of the list_selected_row[i-th]
  # e.g. say for i = 469 and index_oob in below loop is 10 then 
  # list_selected_row[10] (which is an array of row-numbers) should not contain the 469-th row
  for index_oob in range(0, 30):
    if i not in list_selected_row[index_oob]:
      indices_for_oob_models.append(index_oob)
      
  y_predicted_oob_list = []
  
  for oob_model_index in indices_for_oob_models:
    model_oob = list_of_all_models_decision_tree[oob_model_index]
    
    row_oob = x[i]
    # print('oob_model_index ', oob_model_index)
    
    # Now extract ONLY those specific columns/featues that were selected during the bootstrapping
    x_oob_data_point = [row_oob[columns] for columns in list_selected_columns[oob_model_index] ]
    # print('np.array(x_oob_data_point) ', np.array(x_oob_data_point))
    x_oob_data_point = np.array(x_oob_data_point).reshape(1, -1)
    
    y_predicted_oob_data_point = model_oob.predict(x_oob_data_point)
    y_predicted_oob_list.append(y_predicted_oob_data_point)
    # 
  y_predicted_oob_list = np.array(y_predicted_oob_list)
  
  y_predicted_median = np.median(y_predicted_oob_list)
  y_predicted_oob_median_list.append(y_predicted_median)
  

def calculate_oob_score(num_rows):
  oob_score = 0
  for i in range(0, num_rows):
    oob_score += ((y[i] - y_predicted_oob_median_list[i] ) ** 2)
  final_oob_score = oob_score/506
  return final_oob_score

print("final_oob_score is ", calculate_oob_score(506))   
  

final_oob_score is  13.081730298143956


### Further notes on above OOB calculation

The key point is that the OOB sample rows were passed through every Decition Treee that did not contain those specific OOB sample rows during the bootstrapping of training data.

OOB error is simply the error on samples that were not seen during training. 

OOB Scoring is very useful when I dont have a large dataset and thereby if I split that dataset into training and validation set - will result in loss of useful data that otherwise could have been used for training the models. Hence in this case, we decide to extract some of the training data as the validation set by using only those data-points that were not used for training a particular sample-set.

---

# Task 2

* Computing CI of OOB Score and Train MSE
* Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score
* After this we will have 35 Train MSE values and 35 OOB scores
* using these 35 values (assume like a sample) find the confidence intravels of MSE and OOB Score
* we need to report CI of MSE and CI of OOB Score
* Note: Refer the Central_Limit_theorem.ipynb to check how to find the confidence intravel


In [11]:
# Function to build the entire bootstrapping steps that we did above and
# Reurning from the function the MSE and oob score
def bootstrapping_and_oob(x, y):

  # Use generating_samples function to create 30 samples 
  # store these created samples in a list
  list_input_data =[]
  list_output_data =[]
  list_selected_row= []
  list_selected_columns=[]
  
  for i in range (0, 30):
    a, b, c, d = generating_samples(x, y)
    list_input_data.append(a)
    list_output_data.append(b)
    list_selected_row.append(c)
    list_selected_columns.append(d)
  
  # building regression trees
  list_of_all_models_decision_tree = []
  for i in range(0, 30):
    model_i = DecisionTreeRegressor(max_depth=None)
    model_i.fit(list_input_data[i], list_output_data[i])
    list_of_all_models_decision_tree.append(model_i)
  
  # calculating MSE
  array_of_Y = []

  for i in range(0, 30):
    data_point_i = x[:, list_selected_columns[i]]
    target_y_i = list_of_all_models_decision_tree[i].predict(data_point_i)
    array_of_Y.append(target_y_i)
    
    
  predicted_array_of_target_y = np.array(array_of_Y)
  predicted_array_of_target_y = predicted_array_of_target_y.transpose()

  # print(predicted_array_of_target_y.shape)

  # Now to calculate MSE, first calculate the Median of Predicted Y
  # passing axis=1 will make sure the medians are computed along axis=1
  median_predicted_y = np.median(predicted_array_of_target_y, axis=1)
  
  # And now the final MSE
  MSE = mean_squared_error(y, median_predicted_y )
  
  # Calculating OOB Score
  y_predicted_oob_median_list = []

  for i in range(0, 506):
    indices_for_oob_models = []
    
    # For each of i-th row I shall build a list of sample size 30
    # ONLY condition being that this ith row should not be part of
    # the list_selected_row
    for index_oob in range(0, 30):
      if i not in list_selected_row[index_oob]:
        indices_for_oob_models.append(index_oob)
        
    y_predicted_oob_list = []
    
    for oob_model_index in indices_for_oob_models:
      model_oob = list_of_all_models_decision_tree[oob_model_index]
      
      row_oob = x[i]
      # print('oob_model_index ', oob_model_index)
      
      x_oob_data_point = [row_oob[col] for col in list_selected_columns[oob_model_index] ]
      # print('np.array(x_oob_data_point) ', np.array(x_oob_data_point))
      x_oob_data_point = np.array(x_oob_data_point).reshape(1, -1)
      
      y_predicted_oob_data_point = model_oob.predict(x_oob_data_point)
      y_predicted_oob_list.append(y_predicted_oob_data_point)
      # 
    y_predicted_oob_list = np.array(y_predicted_oob_list)
    
    y_predicted_median = np.median(y_predicted_oob_list)
    y_predicted_oob_median_list.append(y_predicted_median)
    

  oob_score = 0

  for i in range(0, 506):
    # oob_score = (oob_score + (y[i] - y_predicted_oob_median_list[i] ) ** 2)
    # 13.828377285079045
    oob_score += (y[i] - y_predicted_oob_median_list[i] ) ** 2

  final_oob_score = oob_score/506
  
  return MSE, final_oob_score

print(bootstrapping_and_oob(x, y))
  

(0.05013239090102449, 14.059993831916806)


In [12]:
import scipy

x=boston.data #independent variables
y=boston.target #target variable

mse_boston_35_times_arr = []
oob_score_boston_35_times_arr = []

# Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score
for i in range(0, 35):
  mse, oob_score = bootstrapping_and_oob(x, y)
  mse_boston_35_times_arr.append(mse)
  oob_score_boston_35_times_arr.append(oob_score)


mse_boston_35_times_arr = np.array(mse_boston_35_times_arr)
oob_score_boston_35_times_arr = np.array(oob_score_boston_35_times_arr)

confidence_level = 0.95
degrees_of_freedom = 34 # sample.size - 1

mean_of_sample_mse_35 = np.mean(mse_boston_35_times_arr)
standard_error_of_sample_mse_35 = scipy.stats.sem(mse_boston_35_times_arr)


# Per document - https://www.kite.com/python/answers/how-to-compute-the-confidence-interval-of-a-sample-statistic-in-python
# confidence_interval = scipy.stats.t.interval(confidence_level, degrees_freedom, sample_mean, sample_standard_error)
confidence_interval_mse_35 = scipy.stats.t.interval(confidence_level, degrees_of_freedom, mean_of_sample_mse_35, standard_error_of_sample_mse_35 )
print("confidence_interval_mse_35 ", confidence_interval_mse_35)

# Now calculate confidence inter for oob score
mean_of_sample_oob_score_35 = np.mean(oob_score_boston_35_times_arr)
standard_error_of_sample_oob_score_35 = scipy.stats.sem(oob_score_boston_35_times_arr)

confidence_interval_oob_score_35 = scipy.stats.t.interval(confidence_level, degrees_of_freedom, mean_of_sample_oob_score_35, standard_error_of_sample_oob_score_35 )
print("confidence_interval_oob_score_35 ", confidence_interval_oob_score_35)


confidence_interval_mse_35  (0.09294459920700518, 0.18099695278943642)
confidence_interval_oob_score_35  (13.768617241526412, 14.885877160705009)


## Observation / Interpretation of above Confidence Interval

By definition we know the interpretation of a 95% confidence interval for the population mean as  -  If repeated random samples were taken and the 95% confidence interval was computed for each sample, 95% of the intervals would contain the population mean.

So in this case

- MSE - There is a 95% chance that the confidence interval of (0.05732086175441538, 0.11667646077442519) contains the true population mean of MSE.
- OOB Score - There is a 95% chance that the confidence interval of (13.274222499705303, 14.427942855729313) contains the true population mean of OOB Score.

# Task 3 (send query point "xq" to 30 models)

## We created 30 models by using 30 samples in TASK-1. Here, we need send query point "xq" to 30 models and perform the regression on the output generated by 30 models

## Flowchart for Task 3

![alt text](https://i.imgur.com/Y5cNhQk.png)

In [13]:
def predict_y_given_x_bootstrap(x_query):
  y_predicted_array_30_sample = []
  
  for i in range(0, 30):
    model_i = list_of_all_models_decision_tree[i]
    # Extract x for ith data point with specific number of featues from list_selected_columns
    x_data_point_i = [x_query[column] for column in list_selected_columns[i]]
    x_data_point_i = np.array(x_data_point_i).reshape(1, -1)
    y_predicted_i = model_i.predict(x_data_point_i)
    y_predicted_array_30_sample.append(y_predicted_i)
    
  y_predicted_array_30_sample = np.array(y_predicted_array_30_sample)
  y_predicted_median = np.median(y_predicted_array_30_sample)
  return y_predicted_median


xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60] 
y_predicted_for_xq = predict_y_given_x_bootstrap(xq)
y_predicted_for_xq    

18.7